<a href="https://colab.research.google.com/github/MattPfeiffer85/OVFAcadamy/blob/main/NewCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q kaggle
from google.colab import files
files.upload()
#create a kaggle folder
!mkdir ~/.kaggle
# Go on kaggle > Account > Create New API token
# Save the json file in your laptop in a dedicated folder
# copy the kaggle.json to folder created
!cp kaggle.json ~/.kaggle
#permission for the json to act
!chmod 600 ~/.kaggle/kaggle.json
# Datasets available here: 
# https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009
!kaggle datasets download -d uciml/red-wine-quality-cortez-et-al-2009
!unzip red-wine-quality-cortez-et-al-2009.zip
#### End of the script to get the data directly from kaggle through colab
# Remember that when you run it, you have to click on the button Choose Files and 
# then select the kaggle.json file from your computer
! kaggle datasets download -d artemkorottchenko/large-boatyacht-pricing-dataset
! unzip large-boatyacht-pricing-dataset.zip

Saving kaggle.json to kaggle.json
  0% 0.00/25.6k [00:00<?, ?B/s]
100% 25.6k/25.6k [00:00<00:00, 22.5MB/s]
Archive:  red-wine-quality-cortez-et-al-2009.zip
  inflating: winequality-red.csv     
  0% 0.00/6.22M [00:00<?, ?B/s]
100% 6.22M/6.22M [00:00<00:00, 106MB/s]
Archive:  large-boatyacht-pricing-dataset.zip
  inflating: boat_dataset.csv        


In [9]:
import pandas as pd
import io
import numpy as np
pd.options.display.max_columns = 100
pd.options.display.max_rows = 50

df = pd.read_csv(io.StringIO(open('/content/boat_dataset.csv', 'rb').read().decode('utf-8', errors='ignore')), sep=",")


In [10]:
## Add element in the database for EDUCATIONAL PURPOSE ONLY
# Add random date in order to do a proper temporal split
df['Advertisement Date'] = pd.to_datetime(df['Advertisement Date'], format='%d.%m.%Y')
#Here is a function that will automatically create a feature of random dates between a defined interval
def random_datetimes_or_dates(start, end, out_format='datetime', n=10): 

    '''   
    unix timestamp is in ns by default. 
    I divide the unix time value by 10**9 to make it seconds 
    (or 24*60*60*10**9 to make it days).
    The corresponding unit variable is passed to 
    the pd.to_datetime function. 
    Values for the (divide_by, unit) pair to select is defined by 
    the out_format parameter.
    for 1 -> out_format='datetime'
    for 2 -> out_format=anything else
    '''
    (divide_by, unit) = (10**9, 's') if out_format=='datetime' else (24*60*60*10**9, 'D')

    start_u = start.value//divide_by
    end_u = end.value//divide_by

    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit=unit)


np.random.seed(893717398)
# Set parameters
d_start = pd.to_datetime('2020-01-01')
d_end = pd.to_datetime('2021-12-31')

# Delete initial Advertisement Date -> if you want to keep the inital dates, don't run it
df.drop(columns=['Advertisement Date'], inplace=True)

# Run function
df['Advertisement Date'] = random_datetimes_or_dates(d_start, d_end, out_format='datetime', n=len(df))

# Keep the date and remove the time
df['Advertisement Date']= df['Advertisement Date'].dt.date

# Parse the date
df['Advertisement Date'] = pd.to_datetime(df['Advertisement Date'], format='%Y-%m-%d')

# Impute Advertissement Date by the the created one when they are missing
#df['Advertisement Date'] = np.where(df['Advertisement Date'].isna(), df['date2'], df['Advertisement Date'])

# Delete the random date created
#df.drop(columns=['date2'], inplace=True)

# Create duplicates
temp = df.sample(n=500, random_state=893717398)
df = df.append(temp)
del(temp)
df.reset_index(drop=True, inplace=True)

df['Advertisement Date'] = df['Advertisement Date'].astype(str)


In [11]:
df.head()

,Price,Category,Boat Type,Manufacturer,Model,Boat name,Type,Year Built,Condition,Length,Width,Depth,Displacement,CE Design Category,Cert Number of People,Number of Cabins,Number of beds,Hull Color,Number of Toilets,Number of Bathrooms,Number of Showers,Material,Fresh Water Cap,Holding Tank,Propulsion,Engine,Engine Performance,Fuel Capacity,Fuel Type,Engine Hours,Max Speed,Cruising Speed,Location,Number of views last 7 days,Comments,Additional Comments,Equipment,Advertisement Date
0,"CHF 3.337,-",Power Boats,Motor Yacht,Rigiflex power boats,CAP 400,NaN,new boat from stock,2017.0,as new,4.00 m,1.90 m,NaN,NaN,NaN,7.0,NaN,NaN,white,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Switzerland » Lake Geneva » Vésenaz,226,NaN,Possibility to arrange it with a complete luxu...,NaN,2021-08-11
1,"EUR 3.490,-",Power Boats,Center console boat,Terhi power boats,400 C,NaN,new boat from stock,2020.0,new,4.00 m,1.50 m,NaN,150 kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thermoplastic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Germany » Bönningstedt,75,"In den Farben weiß oder grün lieferbar.,,2 abs...",NaN,NaN,2020-08-19
2,"CHF 3.770,-",Power Boats,Sport Boat,Marine power boats,370 S - Aluboot,NaN,new boat from stock,NaN,NaN,3.69 m,1.42 m,0.25 m,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Aluminium,NaN,NaN,NaN,(Permission for Lake of Constance),NaN,NaN,NaN,NaN,NaN,NaN,Switzerland » Lake of Zurich » Stäfa ZH,124,NaN,The lightweight and sleek rowing boat for 4 pe...,NaN,2021-01-06
3,"DKK 25.900,-",Power Boats,Sport Boat,Pioner power boats,10 Classic Special Edition,NaN,new boat from stock,2020.0,NaN,3.00 m,1.00 m,NaN,110 kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Denmark » Svendborg,64,"MORSOM OG LETKØRT KVALITETSBÅD!,Nye Pioner 10 ...",NaN,NaN,2020-08-01
4,"SEK 35.000,-",Power Boats,Classic,NaN,Gullholmensnipa 21,NaN,Used boat,1974.0,good,6.30 m,2.50 m,0.75 m,2'000 kg,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inboard with Shaft,VolvoPenta MD 2002,1 x 18 HP / 13 kW,50 l,NaN,500 h,NaN,NaN,Nordsee » Västra Frölunda,131,NaN,NaN,"Anchor,Battery,Bilge Pump,Bilge pump,Cockpit C...",2021-05-05


In [18]:
df.shape
df.dtypes
df.describe(include='all')
df.info()
len(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10844 entries, 0 to 10843
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Price                        10844 non-null  object 
 1   Category                     10844 non-null  object 
 2   Boat Type                    10844 non-null  object 
 3   Manufacturer                 9393 non-null   object 
 4   Model                        10841 non-null  object 
 5   Boat name                    1370 non-null   object 
 6   Type                         10839 non-null  object 
 7   Year Built                   10252 non-null  float64
 8   Condition                    3540 non-null   object 
 9   Length                       10834 non-null  object 
 10  Width                        10779 non-null  object 
 11  Depth                        7575 non-null   object 
 12  Displacement                 5298 non-null   object 
 13  CE Design Catego

10844

In [20]:
df['Year Built'].head()

0    2017.0
1    2020.0
2       NaN
3    2020.0
4    1974.0
Name: Year Built, dtype: float64

In [22]:
# selecting several columns
df[['Year Built', 'Model', 'Category']].head()

,Year Built,Model,Category
0,2017.0,CAP 400,Power Boats
1,2020.0,400 C,Power Boats
2,NaN,370 S - Aluboot,Power Boats
3,2020.0,10 Classic Special Edition,Power Boats
4,1974.0,Gullholmensnipa 21,Power Boats


In [25]:
# selecting rows (works like a filter)
mask1 = df['Year Built'] >2000
mask1
df[mask1].head()

,Price,Category,Boat Type,Manufacturer,Model,Boat name,Type,Year Built,Condition,Length,Width,Depth,Displacement,CE Design Category,Cert Number of People,Number of Cabins,Number of beds,Hull Color,Number of Toilets,Number of Bathrooms,Number of Showers,Material,Fresh Water Cap,Holding Tank,Propulsion,Engine,Engine Performance,Fuel Capacity,Fuel Type,Engine Hours,Max Speed,Cruising Speed,Location,Number of views last 7 days,Comments,Additional Comments,Equipment,Advertisement Date
0,"CHF 3.337,-",Power Boats,Motor Yacht,Rigiflex power boats,CAP 400,NaN,new boat from stock,2017.0,as new,4.00 m,1.90 m,NaN,NaN,NaN,7.0,NaN,NaN,white,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Switzerland » Lake Geneva » Vésenaz,226,NaN,Possibility to arrange it with a complete luxu...,NaN,2021-08-11
1,"EUR 3.490,-",Power Boats,Center console boat,Terhi power boats,400 C,NaN,new boat from stock,2020.0,new,4.00 m,1.50 m,NaN,150 kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thermoplastic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Germany » Bönningstedt,75,"In den Farben weiß oder grün lieferbar.,,2 abs...",NaN,NaN,2020-08-19
3,"DKK 25.900,-",Power Boats,Sport Boat,Pioner power boats,10 Classic Special Edition,NaN,new boat from stock,2020.0,NaN,3.00 m,1.00 m,NaN,110 kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Denmark » Svendborg,64,"MORSOM OG LETKØRT KVALITETSBÅD!,Nye Pioner 10 ...",NaN,NaN,2020-08-01
5,"EUR 3.399,-",Power Boats,Fishing Boat,Linder power boats,355 Sportsman,NaN,new boat from stock,2019.0,new,3.55 m,1.46 m,NaN,84 kg,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Aluminium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Germany » Bayern » München,58,NaN,"For engines up to Type 8, the environmentally ...",NaN,2020-02-03
9,"EUR 3.333,-",Power Boats,Fishing Boat,Crescent power boats,364 Rodd 2.5 Pack,NaN,new boat from stock,2019.0,NaN,3.64 m,1.37 m,NaN,77 kg,NaN,NaN,NaN,NaN,white white,NaN,NaN,NaN,NaN,NaN,NaN,"Outboard, four-stroke",Suzuki DF 2.5,1 x 2 HP / 1.5 kW,NaN,NaN,NaN,NaN,NaN,Germany » Bayern » Boote+service Oberbayern,45,NaN,"together with 2 wooden rudders, 3 wooden bench...",Full Enclosure,2021-11-22


In [26]:
df[mask1].min()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


Price                    CHF 1.060.000,-
Category                     Power Boats
Boat Type                       Bowrider
Year Built                        2001.0
Cert Number of People                1.0
Number of Cabins                    -1.0
Number of beds                     -23.0
Number of Toilets                    1.0
Number of Bathrooms                  1.0
Number of Showers                    1.0
Advertisement Date            2020-01-01
dtype: object

In [27]:
df[df['Year Built'] > 2000].head()

,Price,Category,Boat Type,Manufacturer,Model,Boat name,Type,Year Built,Condition,Length,Width,Depth,Displacement,CE Design Category,Cert Number of People,Number of Cabins,Number of beds,Hull Color,Number of Toilets,Number of Bathrooms,Number of Showers,Material,Fresh Water Cap,Holding Tank,Propulsion,Engine,Engine Performance,Fuel Capacity,Fuel Type,Engine Hours,Max Speed,Cruising Speed,Location,Number of views last 7 days,Comments,Additional Comments,Equipment,Advertisement Date
0,"CHF 3.337,-",Power Boats,Motor Yacht,Rigiflex power boats,CAP 400,NaN,new boat from stock,2017.0,as new,4.00 m,1.90 m,NaN,NaN,NaN,7.0,NaN,NaN,white,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Switzerland » Lake Geneva » Vésenaz,226,NaN,Possibility to arrange it with a complete luxu...,NaN,2021-08-11
1,"EUR 3.490,-",Power Boats,Center console boat,Terhi power boats,400 C,NaN,new boat from stock,2020.0,new,4.00 m,1.50 m,NaN,150 kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thermoplastic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Germany » Bönningstedt,75,"In den Farben weiß oder grün lieferbar.,,2 abs...",NaN,NaN,2020-08-19
3,"DKK 25.900,-",Power Boats,Sport Boat,Pioner power boats,10 Classic Special Edition,NaN,new boat from stock,2020.0,NaN,3.00 m,1.00 m,NaN,110 kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Denmark » Svendborg,64,"MORSOM OG LETKØRT KVALITETSBÅD!,Nye Pioner 10 ...",NaN,NaN,2020-08-01
5,"EUR 3.399,-",Power Boats,Fishing Boat,Linder power boats,355 Sportsman,NaN,new boat from stock,2019.0,new,3.55 m,1.46 m,NaN,84 kg,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Aluminium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Germany » Bayern » München,58,NaN,"For engines up to Type 8, the environmentally ...",NaN,2020-02-03
9,"EUR 3.333,-",Power Boats,Fishing Boat,Crescent power boats,364 Rodd 2.5 Pack,NaN,new boat from stock,2019.0,NaN,3.64 m,1.37 m,NaN,77 kg,NaN,NaN,NaN,NaN,white white,NaN,NaN,NaN,NaN,NaN,NaN,"Outboard, four-stroke",Suzuki DF 2.5,1 x 2 HP / 1.5 kW,NaN,NaN,NaN,NaN,NaN,Germany » Bayern » Boote+service Oberbayern,45,NaN,"together with 2 wooden rudders, 3 wooden bench...",Full Enclosure,2021-11-22


In [28]:
mask1.dtypes

dtype('bool')

In [29]:
mask2 = df['Year Built'].between(2001,2005)
df[mask2].head(2)

,Price,Category,Boat Type,Manufacturer,Model,Boat name,Type,Year Built,Condition,Length,Width,Depth,Displacement,CE Design Category,Cert Number of People,Number of Cabins,Number of beds,Hull Color,Number of Toilets,Number of Bathrooms,Number of Showers,Material,Fresh Water Cap,Holding Tank,Propulsion,Engine,Engine Performance,Fuel Capacity,Fuel Type,Engine Hours,Max Speed,Cruising Speed,Location,Number of views last 7 days,Comments,Additional Comments,Equipment,Advertisement Date
14,"EUR 3.500,-",Power Boats,Sport Boat,GS Nautica power boats,470 OPEN,NaN,Used boat,2004.0,good,4.70 m,2.00 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Italy » Lake Garda » Moniga del Garda (BS),69,"Decriptino:,Small open console boat with winds...",NaN,NaN,2021-11-07
79,"CHF 8.400,-",Power Boats,"Fishing Boat,Sport Boat",Buster power boats,M,NaN,", ,Used boat,Unleaded",2002.0,very good,4.60 m,1.80 m,0.20 m,300 kg,C - Inshore,5.0,NaN,NaN,NaN,NaN,NaN,NaN,Aluminium,NaN,NaN,"Outboard, four-stroke",Yamaha F40BETL,1 x 40 HP / 29 kW,37 l,Unleaded,NaN,NaN,NaN,Switzerland » Bielersee » Le Landeron,NaN,NaN,A perfect boat for every occasion such as excu...,"Anchor,Battery,Bilge Pump,Bimini Top,Fire Exti...",2021-02-24


In [31]:
df[mask2][['Category','Fuel Type']]

,Category,Fuel Type
14,Power Boats,NaN
79,Power Boats,Unleaded
91,Power Boats,Unleaded
103,Power Boats,Unleaded
108,Power Boats,Unleaded
...,...,...
10808,Power Boats,NaN
10809,Power Boats,Diesel
10812,Power Boats,Diesel
10824,Power Boats,Diesel


In [42]:
#LOGICAL EXPRESSIONS
mask3 = ((df['Year Built']<=2001) | (df['Year Built']>=2011)) & ((df['Number of Cabins'] <=2) | (df['Number of Cabins'].isna())) & (df['Fuel Type'] == 'Diesel')
df[mask3].head()

,Price,Category,Boat Type,Manufacturer,Model,Boat name,Type,Year Built,Condition,Length,Width,Depth,Displacement,CE Design Category,Cert Number of People,Number of Cabins,Number of beds,Hull Color,Number of Toilets,Number of Bathrooms,Number of Showers,Material,Fresh Water Cap,Holding Tank,Propulsion,Engine,Engine Performance,Fuel Capacity,Fuel Type,Engine Hours,Max Speed,Cruising Speed,Location,Number of views last 7 days,Comments,Additional Comments,Equipment,Advertisement Date
21,"EUR 4.000,-",Power Boats,Pilothouse,Bénéteau power boats,ANTARES 750 IB,L'ANTONIN,"Used boat,Diesel",1981.0,NaN,8.00 m,2.84 m,0.80 m,NaN,NaN,NaN,1.0,4.0,White,NaN,NaN,NaN,PVC,NaN,NaN,NaN,RENAULT COUACH,1 x 145 HP / 107 kW,160 l,Diesel,2850 h,NaN,NaN,"France » MARTGUES, MARTIGUES",265,Une légendaire robustesse de construction : Co...,NaN,NaN,2021-10-21
68,"EUR 6.500,-",Power Boats,Cabin Boat,Cranchi power boats,C 6 (Diesel),NaN,"Used boat,Diesel",1990.0,NaN,6.28 m,2.27 m,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,GRP,NaN,NaN,NaN,VM Motori 170 TD,1 x 170 HP / 125 kW,NaN,Diesel,300 h,NaN,NaN,Italy » Venezia,85,"Domestic Facilities onboard:,Manual Toilet.,,U...",NaN,"Battery,Generator",2020-09-21
86,"EUR 7.500,-",Power Boats,Cabin Boat,NaN,Motorkruiser M&M 8.50,NaN,"Used boat,Diesel",1971.0,NaN,8.50 m,2.20 m,0.65 m,5000 kg,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,Steel,NaN,NaN,NaN,5720 h,NaN,NaN,Diesel,5720 h,NaN,NaN,Netherlands » Sneek,264,Neatly maintained motor cruiser Molenmaker & M...,NaN,"Battery Charger,Depth Instrument,Fire Extingui...",2020-12-26
123,"EUR 9.500,-",Power Boats,Motor Yacht,Cox Marine power boats,Cox 22,NaN,"Used boat,Diesel",1985.0,very good,6.80 m,2.50 m,0.60 m,NaN,NaN,NaN,1.0,NaN,blue,1.0,NaN,1.0,GRP,100 l,NaN,Inboard with Shaft,Yanmar 2GM20F,NaN,60 l,Diesel,NaN,NaN,NaN,Germany » Brandenburg » Berlin / Brandenburg,162,Die Cox ist ein englischer Klassiker und hat m...,NaN,NaN,2020-12-29
137,"EUR 10.000,-",Power Boats,Working Boat,Costruzioni Navali Srl power boats,Rimorchiatore,NaN,"Used boat,Diesel",1975.0,good,12.00 m,3.00 m,NaN,NaN,NaN,NaN,NaN,NaN,grey Rosso,NaN,NaN,NaN,Steel,NaN,NaN,NaN,FIAT 521,1 x 160 HP / 118 kW,NaN,Diesel,NaN,NaN,NaN,Italy » Lombardia - Trentino Alto Adige » Cremona,182,"Used tugboat called ""Levante"", year 1975.,,Cha...",NaN,NaN,2020-04-06


In [43]:
list_cat_to_keep = ['Diesel','Electric']
# is in a list
mask5 = ((df['Year Built']<=2001) | (df['Year Built']>=2011)) & (df['Number of Cabins'] <=2) & (df['Fuel Type'].isin(list_cat_to_keep))

In [45]:
df[mask5]['Fuel Type'].unique()

array(['Diesel', 'Electric'], dtype=object)

In [46]:
# Is not in
mask6 = ((df['Year Built']<=2001) | (df['Year Built']>=2011)) & (df['Number of Cabins'] <=2) & (~df['Fuel Type'].isin(list_cat_to_keep))
df[mask6]['Fuel Type'].unique()

array(['Unleaded', nan, 'Hybrid', 'Gas'], dtype=object)